## Plotting stars on StarTracker images - 18-Nov-22

Craig Lage

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import astropy.io.fits as pf
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
%matplotlib inline

import lsst.afw.image as afwImage
from lsst.geom import SpherePoint
from lsst.geom import Angle as afwAngle
from astroquery.simbad import Simbad
from lsst.summit.utils.blindSolving import runImchar, genericCameraHeaderToWcs, CommandLineSolver

In [ ]:
solver = CommandLineSolver(indexFiles='/scratch/mfisherlevine/astrometry_net/broiler.astrometry.net/~dstn/4100',
                           checkInParallel=True)

path = '/project/GenericCamera/102/2022/11/23/'

In [ ]:
# Get the image and solve for the WCS
seqNum = 9
filename = path + f'GC102_O_20221123_{seqNum:06}.fits'
exp = afwImage.ExposureF(filename)
wcs = genericCameraHeaderToWcs(exp)
exp.setWcs(wcs)
results = runImchar(exp, 5, 25)
newWcs = solver.run(exp, results.sourceCat, silent=True)
raSolve = newWcs.getSkyOrigin().getRa().asDegrees()
decSolve = newWcs.getSkyOrigin().getDec().asDegrees()
"""
# Now find objects within the image
r=3.0
mag_limit = 0.0
mag_range = 8.0
criteria = (f"region(circle,ICRS,{raSolve} {decSolve},{r}d) & "
            f"Vmag > {mag_limit} & Vmag < {mag_limit+mag_range} & "
            "cat = HD")
results = Simbad.query_criteria(criteria)
print(results)
xs = []
ys = []
names = []
for i in range(len(results)):
    ra = Angle(results[i][1] + 'hours')
    dec = Angle(results[i][2] + 'degrees')
    spherePoint = SpherePoint(afwAngle(ra.rad), afwAngle(dec.rad))
    point = newWcs.skyToPixel(spherePoint)
    xs.append(point.x)
    ys.append(point.y)
    name = results[i][0]
    names.append(name)
"""

In [ ]:
# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.title("StarTracker Wide Field = 20221115 seqNo 28")
img = plt.imshow(exp.image.array,  interpolation='Nearest', cmap='gray')
plt.scatter(xs, ys\
            ,facecolors='none', edgecolors='g', s=200, lw=2)
for x, y, name in zip(xs, ys, names):
    plt.text(x+50,y, name, color='g')
colorbar(img)
plt.tight_layout(h_pad=1)
#plt.savefig("/home/craiglagegit/DATA/StarTracker_Labels_Simbad_20221115_28.pdf")

In [ ]:
print("SeqNum RA(header) Dec(header) RA(Solve) Dec(Solve) Az(hdr-fix) El(header) Az(solve) El(solve) DeltaAz DeltaEl Rot\n")
hdulist = pf.open(filename)
header = hdulist[0].header
raStart = header["RASTART"]
decStart = header["DECSTART"]
azStart = header["AZSTART"]
azStart = 360.0 - azStart # Compensating for Az sign error
elStart = header["ELSTART"]
rotpa = header["ROTPA"]
rot = newWcs.getRelativeRotationToWcs(wcs).asDegrees()
time = Time(header["DATE-OBS"])
altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
skyLocation = SkyCoord(raSolve * u.deg, decSolve * u.deg)
obsAltAz = skyLocation.transform_to(altAz)
az = obsAltAz.az.deg
el = obsAltAz.alt.deg
deltaAz = azStart - az
deltaEl = elStart - el
print(f"{seqNum}      {raStart:.3f}    {decStart:.3f}      {raSolve:.3f}    {decSolve:.3f}    {azStart:.3f}      {elStart:.3f}    {az:.3f}    {el:.3f}   {deltaAz:.3f}   {deltaEl:.3f}  {rot:.3f}\n")
